# 🤖 Unified Semantic Search: PDF Profile + CSV Funding + LLM Draft

In [16]:
import sys
!{sys.executable} -m pip install -q openai tiktoken python-dotenv tqdm pandas

In [17]:
import sys
!{sys.executable} -m pip uninstall -y pinecone-client pinecone
!{sys.executable} -m pip install pinecone

Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0
  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


In [19]:
import sys
print(sys.executable)
!{sys.executable} -m pip list | grep pinecone


/Users/kiranmulawad/AI-Funding/.venv/bin/python
langchain-pinecone        0.2.11
pinecone                  7.3.0
pinecone-plugin-assistant 1.7.0
pinecone-plugin-interface 0.0.7


In [21]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import pinecone
from tqdm import tqdm
import fitz  # PyMuPDF
from typing import List
from IPython.display import display

In [ ]:
# # Load env variables
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# PINECONE_ENV = os.getenv("PINECONE_ENV")
# assert OPENAI_API_KEY and PINECONE_API_KEY and PINECONE_ENV

In [ ]:
# # Init Pinecone + OpenAI
# client = OpenAI(api_key=OPENAI_API_KEY)
# pc = Pinecone(api_key=PINECONE_API_KEY)
# index = pc.Index("funding-search")

In [27]:
# Load environment variables
load_dotenv()

True

In [28]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
assert OPENAI_API_KEY and PINECONE_API_KEY and PINECONE_ENV

In [29]:
# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize Pinecone (new 7.x+ style)
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# Connect to existing index or create it if needed
index = pinecone.Index("funding-search")

AttributeError: module 'pinecone' has no attribute 'init'

In [42]:
# # %%
# # Extract PDF profile text
# pdf_path = "/Users/kiranmulawad/AI-Funding/3_openai_model/sample pdf profiles/sample_user_profile.pdf"

# def extract_text_from_pdf(pdf_path: str) -> str:
#     doc = fitz.open(pdf_path)
#     return "\n".join([page.get_text() for page in doc])

# pdf_text = extract_text_from_pdf(pdf_path)
# print("✅ PDF Text Sample:\n", pdf_text[:500])

In [43]:
# # %%
# # Generate a query from the PDF profile
# prompt = f"""You are an expert grant advisor. Based on the following user profile text, generate a single-sentence query to find relevant public funding in Germany:

# Profile:
# \"\"\"
# {pdf_text[:2000]}
# \"\"\"

# Only output the query — do not include explanations or headers."""

# query_response = client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": "You generate funding search queries from user descriptions."},
#         {"role": "user", "content": prompt}
#     ]
# )

# query = query_response.choices[0].message.content.strip()
# print("📝 Auto-generated query:", query)


In [4]:
query = "We are an AI company focused on AI for robotics. We are focusing on research right now."

In [7]:
# %%
# Embed query
query_embedding = client.embeddings.create(
    input=[query],
    model="text-embedding-3-small"
).data[0].embedding

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************eFEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [46]:
# %%
# 🔍 Query CSV-based funding vectors
semantic_matches = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="openai-v3"
)

In [47]:
matches = [match["metadata"] for match in semantic_matches["matches"]]
funding_results_df = pd.DataFrame(matches)

In [48]:
desired_order = [
    "name", "description", "domain", "eligibility", "amount",
    "deadline", "url", "id"
]

# Reorder the DataFrame
funding_results_df = funding_results_df[desired_order]

In [49]:
display(funding_results_df)

,name,description,domain,eligibility,amount,deadline,url,id
0,AIRISE Open Call,AIRISE supports SMEs and mid-caps in the imple...,AI in production,SMEs and mid-caps,"Up to 60,000 euros","July 16, 2025",https://airise.eu/calls,0965271e1a6f6f68548db796c1e7c80d
1,"Funding for projects on the topic of ""Applicat...",Guideline for the funding of projects on the t...,"Research & Innovation (topic-specific), Health...","University, research institution, company, ass...",amount information not found,deadline information not found,https://www.foerderdatenbank.de/FDB/Content/DE...,cd21459049ca19b22a72fffcda329c79
2,FORTIS 1st Open Call,FORTIS supports projects that develop innovati...,"Human-robot interaction (HRI), multimodal comm...","Consortia of 2-3 organizations: Start-ups, SME...","Up to 250,000 euros","June 04, 2025",https://fortis-project.eu/open-call-1/,bd4c4d243afbba1223855b5ab41181a9
3,Promotion of interdisciplinary pilot projects ...,Guideline for the funding of interdisciplinary...,Research & Innovation (topic-specific),"Research institution, university, company",amount information not found,deadline information not found,https://www.foerderdatenbank.de/FDB/Content/DE...,7f11351b7489d381f3c2e474b06d25eb
4,MASTER 2nd Open Call,MASTER supports projects that develop and vali...,VR/XR,"Universities, research institutions, education...","Up to 100,000 euros","June 12, 2025",https://www.master-xr.eu/open-calls/open-call-2/,b0e5c199bfa519c5a1d8c452edd92901


In [ ]:
def generate_structured_funding_blocks(matches, user_query: str) -> str:
    formatted_blocks = []

    field_aliases = {
        "Amount": ["amount", "how much", "funding", "money"],
        "Deadline": ["deadline", "last date", "until", "submission date"],
        "Eligibility": ["eligible", "eligibility", "who can apply"],
        "Procedure": ["procedure", "how to apply", "application", "steps", "process"],
        "Contact": ["contact", "email", "person", "support"],
    }

    for idx, match in enumerate(matches, start=1):
        meta = match["metadata"]
        name = meta.get("name", "Unnamed")

        # Extract all fields
        fields = {
            "Description": meta.get("description"),
            "Domain": meta.get("domain"),
            "Eligibility": meta.get("eligibility"),
            "Amount": meta.get("amount"),
            "Deadline": meta.get("deadline"),
            "Procedure": meta.get("procedure"),
            "Contact": meta.get("contact"),
            "URL": meta.get("url"),
        }

        # Detect missing fields user asked about
        missing_fields = []
        for key, value in fields.items():
            if key in field_aliases:
                if any(alias in user_query.lower() for alias in field_aliases[key]):
                    if not value or "not found" in str(value).lower():
                        missing_fields.append(key)

        # Start formatting block
        block = f"""**{idx}. {name}**\n"""
        for key in ["Description", "Domain", "Eligibility", "Amount", "Deadline", "Procedure", "Contact"]:
            val = fields[key]
            if val and "not found" not in str(val).lower():
                block += f"   - **{key}**: {val}\n"

        # Add missing info statement if needed
        if missing_fields:
            block += f"   - *Couldn't trace information about {', '.join(missing_fields)}.*\n"

        # Add URL
        if fields["URL"]:
            block += f"   - **For more information visit**: {fields['URL']}\n"

        formatted_blocks.append(block)

    return "\n".join(formatted_blocks)


In [ ]:
semantic_output = generate_structured_funding_blocks(semantic_matches["matches"], query)
print(semantic_output)

**1. AIRISE Open Call**
   - **Description**: AIRISE supports SMEs and mid-caps in the implementation and scaling of AI solutions in manufacturing. The Open Calls are aimed at companies that are either conducting their first AI experiments or want to bring existing applications to a higher level of maturity (TRL). Funding is available for projects in areas such as design & engineering, process monitoring, manufacturing operations, supply chain, cyber security and training Further information can be found here
   - **Domain**: AI in production
   - **Eligibility**: SMEs and mid-caps
   - **Amount**: Up to 60,000 euros
   - **Deadline**: July 16, 2025
   - **Link**: https://airise.eu/calls
   - **For more information, visit**: https://airise.eu/calls

**2. Funding for projects on the topic of "Application of artificial intelligence (AI) in drug discovery"**
   - **Description**: Guideline for the funding of projects on the topic of "Application of artificial intelligence (AI) in drug dis

In [ ]:
top_matches = [
    generate_structured_funding_blocks([match]) for match in semantic_matches["matches"]
]

In [ ]:
# llm_prompt = f"""
# The company described itself as:

# "{query}"

# Here are the top 5 most relevant public funding programs in Germany, based on a semantic search match to their needs:

# {semantic_output}

# Now:

# Please write a concise and professional recommendation containing **only the top 2–3 most relevant funding programs** in this format:

# For each recommendation, follow this format exactly:

# 1. <Program Name>  
# Why it fits: <1–2 lines explaining relevance to the company’s domain (or) industry (or) field of work>  
# **Description**: <1–3 sentence summary of what the program funds and its focus>  
# **Eligibility**: <Eligibility>  
# **Amount**: <Amount>  
# **Deadline**: <Deadline>  
# **Next Steps**:  
# - <Step 1> Visit the official call page: <Program URL>   
# - <Step 2> Refer procedures and guidelines 
# - <Step 3 (if needed)>   

# Use simple bullet points under **Next Steps**. Only list the top 2 or 3 programs — not all 5.
# """


In [ ]:
# response = client.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=[
#         {"role": "system", "content": "You are a startup funding advisor in Germany."},
#         {"role": "user", "content": llm_prompt}
#     ]
# )

In [ ]:
# print("\n🧾 GPT Recommendation:\n")
# print(response.choices[0].message.content)


🧾 GPT Recommendation:

1. **FORTIS 1st Open Call**  
Why it fits: Directly targets innovative solutions for human-robot interaction, aligned with your focus on AI for robotics.  
**Amount**: Up to 250,000 euros  
**Deadline**: June 04, 2025  
**Eligibility**: Consortia of 2-3 organizations: Start-ups, SMEs, mid-caps, research institutions, universities  
**Description**: Focuses on developing technologies for enhancing productivity and interaction between humans and robots, integrating digital twins and multimodal communication in industry settings.  
**Next Steps**:  
- Visit the official call page: [FORTIS 1st Open Call](https://fortis-project.eu/open-call-1/)  
- Consider forming or joining a consortium to apply.  
- Review and prepare the required documentation for submission.

2. **MASTER 2nd Open Call**  
Why it fits: Supports the development of XR-based educational tools for robotics, beneficial for training purposes in AI-driven robotics fields.  
**Amount**: Up to 100,000 eur

In [ ]:
llm_prompt = f"""
The company described itself as:

"{query}"

Here are the top 5 most relevant public funding programs in Germany, based on a semantic search match to their needs:

{semantic_output}

Now:

Please write a concise and professional recommendation containing **only the top 2–3 most relevant funding programs** in this format:

Only select the top programs that most directly match the company’s domain, maturity stage (e.g., early-stage research), or funding needs. Ignore entries that are vague or poorly aligned.

For each recommendation, follow this format exactly:

1. <Program Name>  
Why it fits: <1–2 lines explaining relevance to the company’s domain (or) industry (or) field of work>  
**Description**: <1–3 sentence summary of what the program funds and its focus>  
**Eligibility**: <Eligibility>  
**Amount**: <Amount>  
**Deadline**: <Deadline>  
**Contact**: <Contact person, email, or organization>  
**Next Steps**:  
- <Step 1> Visit the official call page: <Program URL>  
- <Step 2> Review application procedures and eligibility guidelines  
- <Step 3> (Optional) Include any additional steps if mentioned in the procedure  

If any field like **Amount**, **Deadline**, **Eligibility**, **Procedure**, or **Contact** is missing, either omit the line or say “Not specified”.

Use simple bullet points under **Next Steps**. Only list the top 2 or 3 programs — not all 5.
"""


NameError: name 'query' is not defined

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are an expert in funding opportunities."},
        {"role": "user", "content": llm_prompt}
    ]
)
print(response.choices[0].message.content)
